# Add Point and Range data
Work with point and range attribute types, including multi-valued points and open-ended ranges.

## Connect to MI
Get a database and table.

In [1]:
from datetime import datetime
from GRANTA_MIScriptingToolkit import granta as mpy

mi = mpy.connect('http://localhost/mi_servicelayer', autologon=True)
db = mi.get_db(db_key='MI_Training')
db.set_unit_system(unit_system='Metric', absolute_temperatures=False)
tab = db.get_table('Composite Design Data')

## Create a new record and path
Define a path in the table from a starting folder (in this case the top level folder) using `path_from()`.
If the path does not exist, the required folders will be created.
Specify `end_node` to create a new **Record** object at the end of the path with that name.

In [2]:
now = datetime.now().strftime("%c")
record_name = 'STK Example 12:{}'.format(now)
record = tab.path_from(None, tree_path=['Epoxy / Glass', '3M, S-Glass Unitape S2/SP381', '[0]'], end_node=record_name)
record, record.parent

(<Record long name:STK Example 12:Thu Jul 15 10:48:07 2021>,
 <Record long name:3M, S-Glass Unitape S2/SP381, [0]>)

## Edit the record's attributes
Fetch point and range attributes for editing.

In [3]:
tab.bulk_fetch([record], attributes=['Test temperature',
                                     '0° tension modulus - measured',
                                     'Resin content',
                                     'Fiber volume'])
test_temperature = record.attributes['Test temperature']
modulus = record.attributes['0° tension modulus - measured']
resin_content = record.attributes['Resin content']
fiber_volume = record.attributes['Fiber volume']

### Point attributes
Assign a list of numeric values to the `points` property.

If multiple values are assigned, you must also assign a list of dictionaries containing the parameter values to the
`parameters` property. (The two lists must be the same length.)
Here, a single parameter called *Basis* is used to discriminate between the two point values.

In [4]:
test_temperature.points = [23]
test_temperature.unit = '°C'

In [5]:
modulus.points = [8, 7.5]
modulus.unit = 'GPa'
modulus.parameters = [{'Basis': 'Mean'}, {'Basis': 'A-basis'}]

### Range attributes
Access the `value` property directly and assign either a dictionary or tuple for high and low values.
Omitting either the 'low' or 'high' value creates an open-ended range.

In [6]:
resin_content.value = {'low': 28, 'high': 30}
resin_content.unit = 'wt%'

In [7]:
fiber_volume.value = (None, 62.0)
fiber_volume.unit = '%'

## Write your changes to MI
First, specify the attributes on the record which you want to update on the server. Then write the changes to MI.
The list of updated **Record** objects is returned.

In [8]:
record.set_attributes([resin_content, test_temperature, fiber_volume, modulus])
record = mi.update([record])[0]

## Output the record's attributes
Access the attribute values via the same properties you used to assign them.

In [9]:
test_temperature = record.attributes['Test temperature']
print("Test temperature: {0} {1}".format(test_temperature.points[0], test_temperature.unit))

modulus = record.attributes['0° tension modulus - measured']
print('0° tension modulus: ', end="")
formatted_points = ["{0} {1} ({2})".format(point, modulus.unit, modulus.parameters[idx]['Basis'])
                    for idx, point in enumerate(modulus.points)]
print(", ".join(formatted_points))

Test temperature: 23.0 °C
0° tension modulus: 7.5 GPa (A-basis), 8.0 GPa (Mean)


In [10]:
resin_content = record.attributes['Resin content']
print("Resin content: {0}-{1} {2}".format(resin_content.value['low'], resin_content.value['high'], resin_content.unit))

fiber_volume = record.attributes['Fiber volume']
print("Fiber volume: < {0} {1}".format(fiber_volume.value['high'], fiber_volume.unit))

Resin content: 28.0-30.0 wt%
Fiber volume: < 62.0 %
